# VOCS v1 data structure 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [3]:
?VOCS

Init signature:
VOCS(
    *,
    variables: Dict[str, types.ConstrainedListValue] = None,
    constraints: Dict[str, types.ConstrainedListValue] = None,
    objectives: Dict[str, xopt.vocs.ObjectiveEnum] = None,
    constants: Dict[str, Any] = None,
    linked_variables: Dict[str, str] = None,
) -> None
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/Code/GitHub/Xopt/xopt/vocs.py
Type:           ModelMetaclass
Subclasses:     


In [4]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables=None)

In [5]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': None}

In [6]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [7]:
# Check that these are the same
vocs2 == vocs


True

In [8]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [9]:
vocs.objectives['c']  == ObjectiveEnum.MAXIMIZE

True

In [10]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": null}'

In [11]:
vocs.schema()

{'title': 'VOCS',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'definitions': {'ConstraintEnum': {'title': 'ConstraintEnum',
   'description': 'An enumeration.',
   'enum': ['LESS_THAN', 'GREATER_THAN'],
   'type': 'string'},
  'ObjectiveEnum': {'title': 'ObjectiveEnum',
   'description': 'An enumeration.',


# Objective Evaluation

In [12]:
from xopt.vocs import objective_data, constraint_data   
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data['c']  = data['a'] + data['b']
data['d']  = data['a'] - data['b']
data['e']  = data['a']*2 + data['b']*2
data['f']  = data['a']*2 - data['b']*2
data.index = np.arange(len(data)) + 5 # custom index
data


,a,b,g,c,d,e,f
5,47.330197,-0.665136,1234,46.665061,47.995333,93.330122,95.990666
6,439.472656,-0.366679,1234,439.105978,439.839335,878.211955,879.678670
7,704.448081,0.898534,1234,705.346615,703.549547,1410.693230,1407.099094
8,711.319684,0.668119,1234,711.987803,710.651565,1423.975606,1421.303130
9,885.584861,-0.375376,1234,885.209485,885.960238,1770.418970,1771.920476
10,447.783647,-0.585419,1234,447.198228,448.369067,894.396456,896.738134
11,988.028448,-0.743927,1234,987.284521,988.772374,1974.569042,1977.544748
12,339.600880,0.082769,1234,339.683649,339.518111,679.367298,679.036223
13,973.873013,0.616529,1234,974.489542,973.256484,1948.979084,1946.512968
14,833.388710,-0.329753,1234,833.058957,833.718463,1666.117915,1667.436926


In [13]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [14]:
?objective_data

Signature: objective_data(vocs, data, prefix='objective_')
Docstring:
Use objective dict and data (dataframe) to generate objective data (dataframe)

Weights are applied to convert all objectives into mimimization form.

Returns a dataframe with the objective data intented to be minimized.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [15]:
# These are in standard form for minimization
objective_data(vocs, data)

,objective_c,objective_d
5,-46.665061,47.995333
6,-439.105978,439.839335
7,-705.346615,703.549547
8,-711.987803,710.651565
9,-885.209485,885.960238
10,-447.198228,448.369067
11,-987.284521,988.772374
12,-339.683649,339.518111
13,-974.489542,973.256484
14,-833.058957,833.718463


In [16]:
# use the to_numpy() method to convert for low level use.
objective_data(vocs, data).to_numpy()

array([[ -46.66506091,   47.995333  ],
       [-439.10597769,  439.83933523],
       [-705.34661495,  703.54954689],
       [-711.98780315,  710.65156496],
       [-885.20948516,  885.96023779],
       [-447.19822786,  448.36906677],
       [-987.28452096,  988.77237416],
       [-339.68364912,  339.51811142],
       [-974.48954205,  973.25648408],
       [-833.05895736,  833.71846296]])

In [17]:
vocs.constraints

{'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}

In [18]:
?constraint_data

Signature: constraint_data(vocs, data, prefix='constraint_')
Docstring:
Use constraint dict and data (dataframe) to generate constraint data (dataframe)
A constraint is satisfied if the evaluation is > 0.

Returns a dataframe with the constraint data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [19]:
constraint_data(vocs, data)

,constraint_e,constraint_f
5,-91.330122,95.990666
6,-876.211955,879.678670
7,-1408.693230,1407.099094
8,-1421.975606,1421.303130
9,-1768.418970,1771.920476
10,-892.396456,896.738134
11,-1972.569042,1977.544748
12,-677.367298,679.036223
13,-1946.979084,1946.512968
14,-1664.117915,1667.436926
